In [3]:
import numpy as np

In [94]:
def probability(w):
    val_1 = np.random.normal([2, 5], [[2,1],[1,2]])
    val_2 = np.random.normal([3,1], [[0.1,0],[0,0.1]])
    val_3 = np.random.normal(w, [[2,1],[1,2]])
    val_4 = np.random.normal(w, [[0.1,0],[0,0.1]])
    return val_1 + val_2 + val_3 + val_4

# Rejection Sampling

In [101]:
print(probability([0.5,0.5]))

[[11.09126223  6.67438776]
 [ 5.56624908  1.58599085]]


# Importance Sampling